In [1]:
%load_ext autoreload
%autoreload 2

import plotly.express as px
import plotly.graph_objects as go
import re
import time
import utils 
import json
import glob
import pandas as pd

from typing import Dict, List, Optional

def read_file_lines(file_path: str):
    try:
        # with open(file_path, 'r') as f:
        with open(file_path, 'r', encoding='utf-8', errors='replace') as f:
            lines = f.readlines()

        return lines
    except Exception as e:
        print(e)
        return None
    
def get_loss_values(job_log):
    lines = read_file_lines(job_log)
    loss_values = []
    
    for line in lines:
        match = re.search(r"'loss': (\d+\.\d+)", line)
        if match:
            loss_value = float(match.group(1))
            loss_values.append(loss_value)
    
    return loss_values

def plot_train_loss(
    loss_dict: Dict[str, List[float]], 
    title: Optional[str] = None, 
    skip_steps: Optional[Dict[str, int]]= None
):
    """
    loss_dict = {
        name: [loss array],
        name2: [loss array2],
        ...
    }
    """
    if skip_steps is None:
        skip_steps = {}
        
    fig = go.Figure()

    for name, loss_array in loss_dict.items():
        skip = skip_steps.get(name, 0)
        steps = [skip + 10*i for i in range(len(loss_array))]
        fig.add_trace(go.Scatter(x=steps, y=loss_array, name=name))

    fig.update_layout(title=f'Train Loss: {name}',
                      xaxis_title='Step',
                      yaxis_title='Loss')
    fig.show()
    
    
def get_loss_from_checkpoint(checkpoint_dir):
    checkpoints = glob.glob(f"{checkpoint_dir}/checkpoint-*")
    max_checkpoint = 0
    for c in checkpoints:
        num = int(c.split('-')[-1])
        max_checkpoint = max(num, max_checkpoint)

    print("max_checkpoint:", max_checkpoint)

    trainer_state = f"{checkpoint_dir}/checkpoint-{max_checkpoint}/trainer_state.json"
    with open(trainer_state, 'r') as f:
        state = json.load(f)

    steps = [s['step'] for s in state['log_history']]
    losses = [s['loss'] for s in state['log_history']]

    return steps, losses

# MM8 Tests

In [ ]:
log_dict = {
    # "70b_8node_yury_6717": "/data/home/yastashonok/logs/output_6717.txt",
    # "70b_64node_harry_6733": "/fsx_0/user/tranx/output/slurm_logs/output_6733.txt",
    # "70b_128node_harry_6743": "/fsx_0/user/tranx/output/slurm_logs/output_6743.txt",
    # "70b_8node_harry_6815_w_BroadcastDataset": "/fsx_0/user/tranx/output/slurm_logs/output_6815.txt",
    # "70b_128node_harry_6839_w_BroadcastDataset": "/fsx_0/user/tranx/output/slurm_logs/output_6839.txt",
    # "70b_160node_harry_6840_w_BroadcastDataset": "/fsx_0/user/tranx/output/slurm_logs/output_6840.txt"
    "70B_160node_AWS": "/fsx_0/user/tranx/output/slurm_logs/output_6840.txt"
}

loss_values = {}
for name, log in log_dict.items():
    print(name, log)
    loss_values[name] = get_loss_values(log)

fig = go.Figure()
for name, losses in loss_values.items():
    steps = list(range(len(losses)))

    if name == "70b_128node_harry_6839_w_BroadcastDataset":
        gap = len(loss_values["70b_128node_harry_6743"])
        steps = [s + gap for s in steps]

    fig.add_trace(go.Scatter(x=steps, y=losses, name=name))

fig.show()

# 70B - fixed pipeline

In [ ]:
runs = {
    "70B_160node_AWS-1": "/fsx_0/user/tranx/output/slurm_logs/output_6840.txt",
    "70B_256node_AWS-2": "/fsx_0/user/tranx/output/slurm_logs/output_6926.txt",
    "70B_256node_AWS-3": "/fsx_0/user/tranx/output/slurm_logs/output_6939.txt"
}

checkpoint_interval = 200

loss_values = {}
for name, log in runs.items():
    print(name, log)
    loss_values[name] = get_loss_values(log)

fig = go.Figure()
resume_step = 0

for name, losses in loss_values.items():
    steps = [resume_step + 10*i for i in range(len(losses))]
    resume_step = steps[-1]
    resume_step = checkpoint_interval * (resume_step // checkpoint_interval)
    fig.add_trace(go.Scatter(x=steps, y=losses, name=name))

fig.update_layout(title='Loss Values',
                  xaxis_title='Steps',
                  yaxis_title='Loss')
fig.show()

In [ ]:
runs = [
    "/fsx_0/user/tranx/output/slurm_logs/output_6840.txt",
    "/fsx_0/user/tranx/output/slurm_logs/output_6926.txt",
    "/fsx_0/user/tranx/output/slurm_logs/output_6939.txt"
]

aws_loss = []
for log in runs:
    loss = get_loss_values(log)
    aws_loss.extend(loss)
aws_steps = [10*i for i in range(len(aws_loss))]

In [ ]:
df = pd.read_csv("~/train_loss_f578009631.csv")
fbl_steps = df.step
fbl_loss = df.loss

In [ ]:
# fbl_steps = [10, 70, 800, 1000, 2000]  # , 4000]
# fbl_loss = [9.581, 2.498, 1.348, 1.225, 1.121]  # , 1.065]

fig = go.Figure()

fig.add_trace(go.Scatter(x=aws_steps, y=aws_loss, name="AWS"))
fig.add_trace(go.Scatter(x=fbl_steps, y=fbl_loss, name="FBL"))
# fig.add_trace(go.Scatter(x=fbl_steps, y=fbl_loss, name="FBL",
#               mode="markers", marker_symbol="star", marker_size=10))

fig.update_layout(
    title='AWS vs. FBL comparison: 70B MM8 train loss',
    xaxis_title='Steps',
    yaxis_title='Loss',
    legend=dict(orientation="h", yanchor="bottom", y=1.02)
)
fig.show()

# MM9

## Initial Tests

In [ ]:
log_dict = {
    "MM9_70B_2nodes": "/fsx_0/user/tranx/output/slurm_logs/output_6955.txt"
}

loss_values = {}
for name, log in log_dict.items():
    print(name, log)
    loss_values[name] = get_loss_values(log)

fig = go.Figure()
for name, losses in loss_values.items():
    steps = [10*i for i in range(len(losses))]

    fig.add_trace(go.Scatter(x=steps, y=losses, name=name))

fig.update_layout(title='Train Loss',
                  xaxis_title='Step',
                  yaxis_title='Loss')
fig.show()

In [ ]:
runs = [6973, 6977]
losses = []
for j in runs:
    log = f"/fsx_0/user/tranx/output/slurm_logs/output_{j}.txt"
    print(f"reading losses from {log}")
    log_losses = get_loss_values(log)
    losses.extend(log_losses)


fig = go.Figure()

steps = [10*i for i in range(len(losses))]

name = "MM9_70B_Llama3.1"
fig.add_trace(go.Scatter(x=steps, y=losses, name="MM9_70B_Llama3.1"))

fig.update_layout(title=f'Train Loss: {name}',
                  xaxis_title='Step',
                  yaxis_title='Loss')
fig.show()

## MM9_70B_Llama3.1_336px_128nodes

In [ ]:
# FBL config

runs = [7024]
losses = []
for j in runs:
    log = f"/fsx_0/user/tranx/output/slurm_logs/output_{j}.txt"
    print(f"reading losses from {log}")
    log_losses = get_loss_values(log)
    losses.extend(log_losses)


plot_train_loss(
    loss_dict={"MM9_70B_Llama3.1": losses},
    title="MM9_70B_Llama3.1_336px_128nodes"
)

## MM9 - all runs

### Old

In [ ]:
schemes = [
    {
        "name": "f587797729_70B_Llama3.1_336px_128nodes_bz64",
        "name_comment": "#5 f587797729_70B_Llama3.1_336px_128nodes_bz64",
        "loss": df['loss'],
        "steps": df['step'],
        "resume_step": 0,
        "step_scale": 10,  # 10 for bz=32, gradient_accumulation_step=4, nodes=128
        "color": "black"
    },
    {
        "name": "MM9_70B_Llama3.1_336px_128nodes_bz32",
        "name_comment": "#1.2 MM9_70B_Llama3.1_336px_128nodes_bz32_retrain",
        # something missing between 8231 and 8390
        # "jobs": [7260, 7499, 8231, 8390, 8407],
        # "jobs": [7260, 7499],
        "loss": lm31_losses,
        "steps": lm31_steps,
        "resume_step": 0,
        "step_scale": 10,  # 10 for bz=32, gradient_accumulation_step=4, nodes=128
        "color": "blue"
    },
    # {
    #     "name": "MM9_70B_MH19_336px",
    #     "name_comment": "#3.2 MM9_70B_MH19_336px_128nodes_fixed",
    #     "jobs": [7144],
    #     "resume_step": 3300,
    #     "step_scale": 10,
    #     "color": "red"
    # },
    # {
    #     "name": "#3.3 MM9_70B_MH19_336px_128nodes_retrain",
    #     "jobs": [7261],
    #     "resume_step": 0,
    #     "step_scale": 10,
    #     "color": "red"
    # },
    # {
    #     "name": "#3.5 MM9_70B_MH19_336px_384nodes",
    #     "jobs": [7343],
    #     "resume_step": 2100,
    #     "step_scale": 30,
    #     "color": "red"
    # },
    # {
    #     "name": "MM9_70B_MH19_336px",
    #     "name_comment": "#3.6 MM9_70B_MH19_336px_256nodes",
    #     "jobs": [7465],
    #     "resume_step": 7500,
    #     "step_scale": 10, #20,
    #     "color": "red"
    # },
    {
        "name": "MM9_70B_MH19_336px",
        "name_comment": "#3.7 MM9_70B_MH19_336px_128nodes_resume",
        "jobs": [7700, 8351],
        "resume_step": 8000,  # 8500,
        "step_scale": 10,
        "color": "red"
    },

]

fig = plot_compare_loss(schemes, yrange=[0.9, 1.4])


fig.show()

### V2

In [2]:
def plot_compare_loss(schemas, yrange=None):

    fig = go.Figure()

    for s in schemas:
        # read loss from jobs
        if "jobs" in s:
            s["loss"] = []
            for j in s["jobs"]:
                if j == 9217:
                    log = f"/fsx_0/user/ahmadyan/logs/pretraining/output_{j}.txt"
                else:
                    log = f"/fsx_0/user/tranx/output/slurm_logs/output_{j}.txt"
                print(f"Reading loss from {log}")
                loss_j = get_loss_values(log)
                s["loss"].extend(loss_j)

        if 'steps' in s:
            steps = s['steps']
        else:
            steps = [s["resume_step"] + i*s["step_scale"]
                     for i in range(len(s["loss"]))]

        if 'color' in s:
            fig.add_trace(go.Scatter(
                x=steps, y=s["loss"], name=s["name"], mode='lines', line=dict(color=s['color'])))
        else:
            fig.add_trace(go.Scatter(
                x=steps, y=s["loss"], name=s["name"], mode='lines'))

    fig.update_layout(
        title=f'Train Loss', xaxis_title='Step', yaxis_title='Loss',
        legend=dict(orientation="h", yanchor="bottom", y=1.02),
        # yaxis=dict(range=[0.8, 1.6])
    )

    if yrange is not None:
        fig.update_layout(yaxis=dict(range=yrange))

    time_ny = utils.get_local_time('America/New_York')
    time_pst = utils.get_local_time('US/Pacific')

    fig.add_annotation(text=f"Last updated (PST): {time_pst}", xref='paper',
                       yref='paper', x=1.0, y=.92, showarrow=False)
    fig.add_annotation(text=f"Last updated (EST): {time_ny}", xref='paper',
                       yref='paper', x=1.0, y=1, showarrow=False)

    return fig

In [4]:
# https://www.internalfb.com/mlhub/pipelines/runs/fblearner/587797729?tab=Visualizations
df = pd.read_csv(
    # "/data/home/tranx/run_516421604076231-4-train_loss.csv",
    # "/data/home/tranx/train_loss_f578009631_0806.csv",
    # "train_loss_f578009631_0806.csv"
    "/fsx_0/user/tranx/experiments/llm_mm_aligner/fb_reference_jobs/train_loss_f578009631_0809.csv",
    header=None, skiprows=1
)
df.columns = ['step', 'loss']
df.head()

fb_31_steps = df['step']
fb_31_loss = df['loss']

lm_dir = "/fsx_0/checkpoints/tranx/MM9-Pretrain-70B/Llama31_336px_128nodes_bz32_scratch"
lm31_steps, lm31_losses = get_loss_from_checkpoint(lm_dir)

mh19_dir = "/fsx_0/checkpoints/tranx/MM9-Pretrain-70B/MH19_336px_128nodes_bz32_resume"
mh19_steps, mh19_losses = get_loss_from_checkpoint(mh19_dir)

mh19_448 = "/fsx_0/checkpoints/ahmadyan/MM9-Pretrain-70B/output_mh19_ve448px_n128"
mh19_448_steps, mh19_448_losses = get_loss_from_checkpoint(mh19_448)

schemes = [
    {
        "name": "f587797729_70B_Llama3.1_336px_128nodes_bz64",
        "name_comment": "#5 f587797729_70B_Llama3.1_336px_128nodes_bz64",
        "loss": df['loss'],
        "steps": df['step'],
        "resume_step": 0,
        # "step_scale": 10,  # 10 for bz=32, gradient_accumulation_step=4, nodes=128
        "color": "black"
    },

    {
        "name": "MM9_70B_MH19_336px_resume",
        "name_comment": "#3.2 MM9_70B_MH19_336px_128nodes_fixed",
        "jobs": [7144],
        "resume_step": 3300,
        "step_scale": 10,
        "color": "red"
    },


    {
        "name": "MM9_70B_MH19_336px_resume",
        "name_comment": "#3.6 MM9_70B_MH19_336px_256nodes",
        "jobs": [7465],
        "resume_step": 7500,
        "step_scale": 10,  # 20,
        "color": "red"
    },

    {
        "name": "MM9_70B_MH19_336px_resume",
        "name_comment": "#3.7 MM9_70B_MH19_336px_128nodes_resume",
        "jobs": [7700, 8351, 9376, 9398],
        "resume_step": 8000,  # 8500,
        "step_scale": 10,
        # "loss": mh19_losses,
        # "steps": mh19_steps,
        "color": "red"
    },

    {
        "name": "MM9_70B_Llama3.1_336px_128nodes_bz32",
        "name_comment": "#1.2 MM9_70B_Llama3.1_336px_128nodes_bz32_retrain",
        "loss": lm31_losses,
        "steps": lm31_steps,
        "resume_step": 0,
        # "step_scale": 10,  # 10 for bz=32, gradient_accumulation_step=4, nodes=128
        "color": "blue"
    },

    # {
    #     "name": "MM9_70B_MH19_448px",
    #     # "loss": mh19_448_losses,
    #     # "steps": mh19_448_steps,
    #     "jobs": [9217],
    #     "step_scale": 5,
    #     "resume_step": 0
    # },

    # {
    #     "name": "#3.3 MM9_70B_MH19_336px_128nodes_retrain",
    #     "jobs": [7261],
    #     "resume_step": 0,
    #     "step_scale": 10,
    #     # "color": "red"
    # },

]

fig = plot_compare_loss(schemes, yrange=[0.8, 1.4])


fig.show()

max_checkpoint: 10200
max_checkpoint: 18700
max_checkpoint: 13800
Reading loss from /fsx_0/user/tranx/output/slurm_logs/output_7144.txt
Reading loss from /fsx_0/user/tranx/output/slurm_logs/output_7465.txt
Reading loss from /fsx_0/user/tranx/output/slurm_logs/output_7700.txt
Reading loss from /fsx_0/user/tranx/output/slurm_logs/output_8351.txt
Reading loss from /fsx_0/user/tranx/output/slurm_logs/output_9376.txt
Reading loss from /fsx_0/user/tranx/output/slurm_logs/output_9398.txt


In [21]:
# publish
utils.save_plotly_to_html(
    fig,
    "/fsx_0/user/tranx/aws_dropbox/AWS_MM9_Pretrain_Stage1_Train_Loss.html"
)

Saved figure to /fsx_0/user/tranx/aws_dropbox/AWS_MM9_Pretrain_Stage1_Train_Loss.html


## Learning rate

In [17]:
checkpoint_dir = "/fsx_0/checkpoints/tranx/MM9-Pretrain-70B/MH19_336px_128nodes_bz32_resume"

checkpoints = glob.glob(f"{checkpoint_dir}/checkpoint-*")
max_checkpoint = 0

for c in checkpoints:
    num = int(c.split('-')[-1])
    max_checkpoint = max(num, max_checkpoint)

print("max_checkpoint:", max_checkpoint)

trainer_state = f"{checkpoint_dir}/checkpoint-{max_checkpoint}/trainer_state.json"
with open(trainer_state, 'r') as f:
    state = json.load(f)

state['log_history']

steps = [s['step'] for s in state['log_history']]
lr = [s['learning_rate'] for s in state['log_history']]

px.line(x=steps, y=lr)

max_checkpoint: 16300


# Train Throughput

In [ ]:
import os


def get_checkpoint_time(dir):
    start_ts = os.path.getctime(os.path.join(dir, "runs"))
    chk_folders = [c for c in os.listdir(dir) if c.startswith('checkpoint')]
    chk_folders

    chk_list = [[int(c.split('-')[-1]), c] for c in chk_folders]
    chk_list = sorted(chk_list)
    chk_list

    for step, folder in chk_list:
        ts = os.path.getctime(os.path.join(dir, folder))
        elasped_ts = ts - start_ts
        start_ts = ts

        print(step, elasped_ts)

In [ ]:
dir160 = '/fsx_0/checkpoints/tranx/Aligner-Pretrain-70B/output_n160_retrain'
dir256 = '/fsx_0/checkpoints/tranx/Aligner-Pretrain-70B/output_n256'
get_checkpoint_time(dir160)
get_checkpoint_time(dir256)

In [ ]:
import numpy as np
t160 = int(np.mean([6737.0, 6805.0, 6970.0]))
t256 = int(np.mean([8526.0, 8497, 8668]))

t160, t256

In [ ]:
time_per_200_steps = t160
nodes = 160

num_images_per_200_steps = 200 * 32 * nodes * 8
num_images_per_day = num_images_per_200_steps * 86400 / time_per_200_steps
mm_images_per_day = num_images_per_day/1e6
int(mm_images_per_day)

In [ ]:
time_per_200_steps = t256
nodes = 256

num_images_per_200_steps = 200 * 32 * nodes * 8
num_images_per_day = num_images_per_200_steps * 86400 / time_per_200_steps
mm_images_per_day = num_images_per_day/1e6
int(mm_images_per_day)

In [ ]:
# fbl per 7 days
days = 7.768
steps = 11840
steps_per_day = steps / days
num_images_per_day = steps_per_day * 32 * 160 * 8
mm_images_per_day = num_images_per_day/1e6
int(mm_images_per_day)

In [ ]:
# fbl per 1st day
days = 1
steps = 1700
steps_per_day = steps / days
num_images_per_day = steps_per_day * 32 * 160 * 8
mm_images_per_day = num_images_per_day/1e6
int(mm_images_per_day)

In [ ]:
num_images = (45739 - 19500) * 32 * 64 * 8  # step x batch_size x nodes x GPUs
trained_time = 24 + 14  # hours
tput_per_day = num_images / (trained_time/24)
tput_M_per_day = tput_per_day / 1e6
print(tput_M_per_day)